# Simple character level tokenizer

In [53]:
with open('shakespeare.txt', 'r') as f:
    text = f.read()
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("\nVocabulary size is ", vocab_size, "chars/tokens: ", ''.join(chars))

ctoi, itoc = dict((char, index) for index, char in enumerate(chars)), dict((index, char) for index, char in enumerate(chars))
print("\nVocabulary: ",  ctoi)

encode = lambda sentence: [ctoi[char] for char in sentence] # encode sentence to index
decode = lambda indexes: ''.join([itoc[index] for index in indexes]) # decode index to sentence
print("example: ",encode("some text for training"), decode(encode("some text for training")))


Vocabulary size is  65 chars/tokens:  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz

Vocabulary:  {'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
example:  [57, 53, 51, 43, 1, 58, 43, 62, 58, 1, 44, 53, 56, 1, 58, 56, 39, 47, 52, 47, 52, 45] some text for training


In [54]:
import torch
data = torch.LongTensor(encode(text))
print("\nData shape:", data.shape, "\nData type:", data.dtype)
print("Data example:\n", data[:])


Data shape: torch.Size([1115393]) 
Data type: torch.int64
Data example:
 tensor([18, 47, 56,  ..., 52, 45,  8])


# Separate data to train and validation 90/10
### generate a small batch of data of inputs x and targets y

In [55]:
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [56]:
# block size
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"input is {context} and target is {target}")

input is tensor([18]) and target is 47
input is tensor([18, 47]) and target is 56
input is tensor([18, 47, 56]) and target is 57
input is tensor([18, 47, 56, 57]) and target is 58
input is tensor([18, 47, 56, 57, 58]) and target is 1
input is tensor([18, 47, 56, 57, 58,  1]) and target is 15
input is tensor([18, 47, 56, 57, 58,  1, 15]) and target is 47
input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) and target is 58


In [57]:
# Batch size dimension
torch.manual_seed(674815) # for reproducibility (same random numbers every time)
batch_size = 4 # number of independent sequences to train on in parallel
block_size = 8 # maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"input is {context.tolist()} and target is {target}")


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [57, 10,  0, 21,  1, 61, 47, 50],
        [44, 43,  1, 40, 63,  1, 58, 46],
        [ 5, 42, 10,  1, 41, 39, 50, 50]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 52],
        [10,  0, 21,  1, 61, 47, 50, 50],
        [43,  1, 40, 63,  1, 58, 46, 43],
        [42, 10,  1, 41, 39, 50, 50, 47]])
----
input is [24] and target is 43
input is [24, 43] and target is 58
input is [24, 43, 58] and target is 5
input is [24, 43, 58, 5] and target is 57
input is [24, 43, 58, 5, 57] and target is 1
input is [24, 43, 58, 5, 57, 1] and target is 46
input is [24, 43, 58, 5, 57, 1, 46] and target is 43
input is [24, 43, 58, 5, 57, 1, 46, 43] and target is 52
input is [57] and target is 10
input is [57, 10] and target is 0
input is [57, 10, 0] and target is 21
input is [57, 10, 0, 21] and target is 1
input is [57, 10, 0, 21, 1] and target is 61
input is [57, 10, 0, 21, 1, 61] and target is 47
input is [57, 

In [58]:
print(xb) # input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [57, 10,  0, 21,  1, 61, 47, 50],
        [44, 43,  1, 40, 63,  1, 58, 46],
        [ 5, 42, 10,  1, 41, 39, 50, 50]])


# Bigram language Model (simplest nn)

In [59]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(674815) # for reproducibility (same random numbers every time)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        # each token directly reads off the logits for the next token from a lookup table
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            # compute the loss
            # loss = F.cross_entropy(logits.view(-1, vocab_size), target.view(-1))
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # 2D tensor for pytorch cross-entropy
            targets = targets.view(B*T) # 1D tensor
            loss = F.cross_entropy(logits, targets)
    
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # appendsampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx
        
    

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, '\nloss is',loss)

print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65]) 
loss is tensor(4.7931, grad_fn=<NllLossBackward0>)

S,cX:?DBL&;Jv:yn?jitTgfMYwsROav
v E
fjDryAu'BiPzjmV'pyYMo

A.ap$JmmkInWxeRRv
GJseP!PgzDUU;Ke.G&tkY;J
